<a href="https://colab.research.google.com/github/elchaima1234/Generating-Sound-Using-Unet-/blob/main/Generating_Sound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**DOWNLOAD THE DATA FORM KAGGLE**

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download mfekadu/darpa-timit-acousticphonetic-continuous-speech

100% 826M/829M [00:21<00:00, 41.8MB/s]
100% 829M/829M [00:21<00:00, 40.2MB/s]


In [6]:
! unzip darpa-timit-acousticphonetic-continuous-speech

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: data/TRAIN/DR6/MTXS0/SA1.WAV  
  inflating: data/TRAIN/DR6/MTXS0/SA1.WAV.wav  
  inflating: data/TRAIN/DR6/MTXS0/SA1.WRD  
  inflating: data/TRAIN/DR6/MTXS0/SA2.PHN  
  inflating: data/TRAIN/DR6/MTXS0/SA2.TXT  
  inflating: data/TRAIN/DR6/MTXS0/SA2.WAV  
  inflating: data/TRAIN/DR6/MTXS0/SA2.WAV.wav  
  inflating: data/TRAIN/DR6/MTXS0/SA2.WRD  
  inflating: data/TRAIN/DR6/MTXS0/SI1060.PHN  
  inflating: data/TRAIN/DR6/MTXS0/SI1060.TXT  
  inflating: data/TRAIN/DR6/MTXS0/SI1060.WAV  
  inflating: data/TRAIN/DR6/MTXS0/SI1060.WAV.wav  
  inflating: data/TRAIN/DR6/MTXS0/SI1060.WRD  
  inflating: data/TRAIN/DR6/MTXS0/SI1690.PHN  
  inflating: data/TRAIN/DR6/MTXS0/SI1690.TXT  
  inflating: data/TRAIN/DR6/MTXS0/SI1690.WAV  
  inflating: data/TRAIN/DR6/MTXS0/SI1690.WAV.wav  
  inflating: data/TRAIN/DR6/MTXS0/SI1690.WRD  
  inflating: data/TRAIN/DR6/MTXS0/SI2320.PHN  
  inflating: data/TRAIN/DR6/MTXS0/SI2

#**Install Denedencies and Setup**

In [7]:
!pip install https://github.com/vBaiCai/python-pesq/archive/master.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pypesq: filename=pypesq-1.2.4-cp310-cp310-linux_x86_64.whl size=89467 sha256=2fa187820950660ccfe4edfffdf8702039d6b1689d5663ac954974da5f57b8b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-u824tezd/wheels/a3/a0/1b/59a1da4e9996e52a180426047716624ebe278be211740388e4
Successfully built pypesq


In [27]:
!pip install pesq

  Preparing metadata (setup.py) ... done
  Created wheel for pesq: filename=pesq-0.0.4-cp310-cp310-linux_x86_64.whl size=262962 sha256=a4be3604bc977b21ae8ce32473343c2ef0a4fb3bf709a86f63a2b9537862d2db
  Stored in directory: /root/.cache/pip/wheels/c5/4e/2c/251524370c0fdd659e99639a0fbd0ca5a782c3aafcd456b28d
Successfully built pesq


In [28]:
import os
import glob
import cv2
from tqdm import tqdm
import librosa
import soundfile as sf
import IPython.display as ipd
import numpy as np
import random
from random import randint
import pypesq
from pesq import pesq
import matplotlib.pyplot as plt
#import pesq
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dropout, LeakyReLU, ReLU, Flatten, Dense, Concatenate, Conv2DTranspose
from tensorflow.keras.activations import sigmoid, relu
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Nadam

# **DOWNLOAD WAV ONLY**

In [9]:
train_path = '/content/data/TRAIN'
test_path = '/content/data/TEST'
wav_list = []
def find_wav_files(folder_path):
  # Loop through all the subfolders in the folder
  for subfolder in tqdm(os.listdir(folder_path)):
    # Join the subfolder name with the folder path
    subfolder_path = os.path.join(folder_path, subfolder)
    # Loop through all the subsubfolders in the subfolder
    for subsubfolder in os.listdir(subfolder_path):
      # Join the subsubfolder name with the subfolder path
      subsubfolder_path = os.path.join(subfolder_path, subsubfolder)
      # Find all the wav files in the subsubfolder using glob
      wav_files = glob.glob(subsubfolder_path + '/*.wav')
      # Extend the wav list with the wav files
      wav_list.extend(wav_files)

find_wav_files(train_path)
find_wav_files(test_path)
print(len(wav_list))

100%|██████████| 8/8 [00:00<00:00, 436.51it/s]

6300


In [10]:
ipd.Audio(wav_list[6])

#**PREPROCESS THE DATA**

**DEFINE PARAMETERS**

In [11]:
sample_rate = 16000
duration = 2
n_fft = 1023
hop_length = 511
window_type = 'hann'
target_shape = (512, 64, 2)
BATCH_SIZE = 16

**resize and add padding**

In [12]:
def pad_or_crop_stft(stft_component, target_shape):
     height_diff = target_shape[0] - stft_component.shape[0]
     width_diff = target_shape[1] - stft_component.shape[1]


     if height_diff > 0:
         stft_component = np.pad(stft_component, ((0, height_diff), (0, 0)), mode='constant')
     else:
         stft_component = stft_component[:target_shape[0], :]


     if width_diff > 0:
         stft_component = np.pad(stft_component, ((0, 0), (0, width_diff)), mode='constant')
     else:
         stft_component = stft_component[:, :target_shape[1]]

     return stft_component

**Convert Data to STFT**

In [13]:
stft_list = []
for audio in wav_list:

    y, sr = librosa.load(audio, sr=sample_rate)
    # # Compute STFT
    stft = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window_type)
    # # Convert STFT to real and imaginary parts(TO ADD CHANNELS)
    stft_real = tf.constant(stft.real, dtype=tf.float32)
    stft_imag = tf.constant(stft.imag, dtype=tf.float32)
    # #======== add padding =======
    # # Pad or crop both the real and imaginary parts of the STFT
    stft_real_padded = pad_or_crop_stft(stft_real, target_shape[:2])
    stft_imag_padded = pad_or_crop_stft(stft_imag, target_shape[:2])
    # #============================
    # # Combine the real and imaginary parts
    stft_combined = tf.stack([stft_real_padded, stft_imag_padded], axis=-1)

    # Normalization

    mean_value = np.mean(stft_combined)
    std_value = np.std(stft_combined)

    # Apply normalization to the entire dataset
    normalized_stft_combined = (stft_combined - mean_value) / std_value
    stft_list.append(normalized_stft_combined)

# **Split the Data**

In [14]:
X_train, X_test  = train_test_split(stft_list, test_size=0.2, random_state=42)

X_train  = np.array(X_train)
X_test  = np.array(X_test)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (5040, 512, 64, 2)
X_test shape: (1260, 512, 64, 2)


# **Build The Model**

In [15]:
def get_embedder():
    SIGNAL_SHAPE = (512, 64, 2)



    inputSTFT=Input(shape = SIGNAL_SHAPE)



    conv_0 = Conv2D(filters=8, kernel_size=5, strides=1, padding='same')(inputSTFT)
    conv_0 = LeakyReLU(alpha=0.2)(conv_0)

    conv_1 = Conv2D(filters=16, kernel_size=5, strides=2, padding='same')(conv_0)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = LeakyReLU(alpha=0.2)(conv_1)

    conv_2 = Conv2D(filters=32, kernel_size=5, strides=2, padding='same')(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = LeakyReLU(alpha=0.2)(conv_2)

    conv_3 = Conv2D(filters=64, kernel_size=5, strides=2, padding='same')(conv_2)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = LeakyReLU(alpha=0.2)(conv_3)

    conv_4 = Conv2D(filters=128, kernel_size=5, strides=2, padding='same')(conv_3)
    conv_4 = BatchNormalization()(conv_4)
    conv_4 = LeakyReLU(alpha=0.2)(conv_4)

    conv_5 = Conv2D(filters=256, kernel_size=5, strides=2, padding='same')(conv_4)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = LeakyReLU(alpha=0.2)(conv_5)

    #bottleneck = Concatenate()([conv_5, inputMessage])

    conv_6 = Conv2D(filters=256, kernel_size=5, strides=1, padding='same', activation=relu)(conv_5)

    upsampling_1 = Conv2DTranspose(filters=128, kernel_size=5, strides=2, padding='same')(conv_6)
    upsampling_1 = Concatenate()([upsampling_1, conv_4])

    conv_7 = BatchNormalization()(upsampling_1)
    conv_7 = ReLU()(conv_7)
    conv_7 = Dropout(rate=0.5)(conv_7)

    upsampling_2 = Conv2DTranspose(filters=64, kernel_size=5, strides=2, padding='same')(conv_7)
    upsampling_2 = Concatenate()([upsampling_2, conv_3])

    conv_8 = BatchNormalization()(upsampling_2)
    conv_8 = ReLU()(conv_8)
    conv_8 = Dropout(rate=0.5)(conv_8)

    upsampling_3 = Conv2DTranspose(filters=32, kernel_size=5, strides=2, padding='same')(conv_8)
    upsampling_3 = Concatenate()([upsampling_3, conv_2])

    conv_9 = BatchNormalization()(upsampling_3)
    conv_9 = ReLU()(conv_9)
    conv_9 = Dropout(rate=0.5)(conv_9)

    upsampling_4 = Conv2DTranspose(filters=16, kernel_size=5, strides=2, padding='same')(conv_9)
    upsampling_4 = Concatenate()([upsampling_4, conv_1])

    conv_10 = BatchNormalization()(upsampling_4)
    conv_10 = ReLU()(conv_10)

    upsampling_5 = Conv2DTranspose(filters=8, kernel_size=5, strides=2, padding='same')(conv_10)
    upsampling_5 = Concatenate()([upsampling_5, conv_0])

    conv_11 = BatchNormalization()(upsampling_5)
    conv_11 = ReLU()(conv_11)


    output = Conv2D(filters=SIGNAL_SHAPE[-1], kernel_size=5, strides=1, padding='same', activation=None)(conv_11)
    return Model(inputs = inputSTFT, outputs = output)

In [16]:
unet = get_embedder()

# **Compile the Model**

In [17]:
unet.compile(optimizer='adam', loss='mean_squared_error', metrics= ['accuracy'])

# **Fit The Model**

In [18]:
unet.fit(X_train, X_train, epochs=20, batch_size=32, validation_data=(X_test, X_test))

Epoch 1/20
158/158 [==============================] - 45s 175ms/step - loss: 0.3137 - accuracy: 0.6024 - val_loss: 0.0930 - val_accuracy: 0.6747
Epoch 2/20
158/158 [==============================] - 19s 120ms/step - loss: 0.0518 - accuracy: 0.6877 - val_loss: 0.0359 - val_accuracy: 0.7182
Epoch 3/20
158/158 [==============================] - 19s 123ms/step - loss: 0.0278 - accuracy: 0.7557 - val_loss: 0.0233 - val_accuracy: 0.7514
Epoch 4/20
158/158 [==============================] - 19s 119ms/step - loss: 0.0195 - accuracy: 0.7873 - val_loss: 0.0173 - val_accuracy: 0.7796
Epoch 5/20
158/158 [==============================] - 19s 120ms/step - loss: 0.0154 - accuracy: 0.8033 - val_loss: 0.0139 - val_accuracy: 0.8048
Epoch 6/20
158/158 [==============================] - 19s 124ms/step - loss: 0.0127 - accuracy: 0.8137 - val_loss: 0.0116 - val_accuracy: 0.8160
Epoch 7/20
158/158 [==============================] - 21s 131ms/step - loss: 0.0107 - accuracy: 0.8207 - val_loss: 0.0098 - val_ac

#**Prediction**

In [20]:
reconstructed_audio = unet.predict(X_test)

40/40 [==============================] - 1s 28ms/step


# **Result Visualisation**

In [21]:
reconstructed_audio[1000].shape

(512, 64, 2)

**Original Input**

In [22]:
original_audio = X_test[1000]
original_audio = (original_audio * std_value) + mean_value
original_stft_resized = tf.complex(original_audio[:,:,0], original_audio[:,:,1])
original_reconstructed_signal = librosa.istft(original_stft_resized.numpy(), hop_length=hop_length, win_length=n_fft-1, center=True)
original_reconstructed_signal = np.squeeze(original_reconstructed_signal)
sf.write("original_audio.wav", original_reconstructed_signal, samplerate=sample_rate)

In [23]:
ipd.Audio('/content/original_audio.wav')

**Reconstructed**

In [24]:
reconstructed_audio = (reconstructed_audio[1000] * std_value) + mean_value

# Inverse STFT
stft_resized = tf.complex(reconstructed_audio[:,:,0], reconstructed_audio[:,:,1])
reconstructed_signal = librosa.istft(stft_resized.numpy(), hop_length=hop_length, win_length=n_fft-1, center=True)
reconstructed_signal = np.squeeze(reconstructed_signal)

# Write the reconstructed audio to a file
sf.write("reconstructed_audio.wav", reconstructed_signal, samplerate=sample_rate)

In [25]:
ipd.Audio('/content/reconstructed_audio.wav')

**PESQ**

In [29]:
from pesq import pesq
from scipy.io import wavfile
rate, ref = wavfile.read('/content/original_audio.wav')
rate, deg = wavfile.read('/content/reconstructed_audio.wav')

print(pesq(rate, ref, deg, 'wb'))
print(pesq(rate, ref, deg, 'nb'))

4.064627647399902
4.18039083480835
